<a href="https://colab.research.google.com/github/kunalsevak24/TrainingLLM/blob/main/LLAMA3-DataTraining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install Ollama on the Colab machine, Run the Ollama server on the colab machine and Pull the llama3 model
!apt-get install lshw
!curl https://ollama.ai/install.sh | sh
!nohup ollama serve &
!ollama pull llama3

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  pci.ids usb.ids
The following NEW packages will be installed:
  lshw pci.ids usb.ids
0 upgraded, 3 newly installed, 0 to remove and 49 not upgraded.
Need to get 790 kB of archives.
After this operation, 2,988 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 lshw amd64 02.19.git.2021.06.19.996aaad9c7-2build1 [321 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 pci.ids all 0.0~2022.01.22-1 [251 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/main amd64 usb.ids all 2022.04.02-1 [219 kB]
Fetched 790 kB in 1s (657 kB/s)
Selecting previously unselected package lshw.
(Reading database ... 123605 files and directories currently installed.)
Preparing to unpack .../lshw_02.19.git.2021.06.19.996aaad9c7-2build1_amd64.deb ...
Unpacking lshw (02.19.git.2021.06.19.996aaad9c7-2build1) ...
Se

In [2]:
!pip install -qU langchain langchain_community faiss-cpu sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 30.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 28.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.3/245.3 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 399.9/399.9 kB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 292.1/292.1 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.9 MB/s eta 0:00:00


In [3]:
from langchain_community.llms.ollama import Ollama
from langchain_community.embeddings.ollama import OllamaEmbeddings
from langchain_community.embeddings.huggingface import HuggingFaceEmbeddings
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain import hub

from pprint import pprint
import bs4

In [4]:
llm = Ollama(model="llama3", temperature=0.7, verbose=True, top_p=1)

# Invoke the model to answer a simple question.
response = llm.invoke("What comes after Monday?")

# print the response to see what the model has generated.
print(response)


That's an easy one!

After Monday, the next day of the week is Tuesday.


In [5]:
# Define a list of URLs that point to the web pages we want to process.
# In this case, the URLs link to stories:
# a) "The Little Match Girl" by Hans Christian Andersen and
# b) "Little Red Riding Hood" from American Literature's website.
urls = [
    "https://kunalsevak24.github.io/",
    "https://www.linkedin.com/in/kunalsevak/"
]

# Create an instance of the WebBaseLoader class.
# This class is used to load textual content from the web pages specified in the `urls` list.
loader = WebBaseLoader(
    web_paths=(urls)
)

# Load the web pages and store the content in the `web_pages` variable.
# This method fetches the HTML content of each URL and typically extracts the text.
web_pages = loader.load()

# Initialize a RecursiveCharacterTextSplitter instance.
# This component is configured to split the text into chunks of 1000 characters with an overlap of 200 characters.
# Chunk size defines how many characters each text chunk will contain.
# Chunk overlap allows for a portion of the text to be repeated in subsequent chunks, which can be useful
# for maintaining context in certain NLP tasks.
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)

# Apply the text splitter to the loaded web pages.
# This will divide the text of each page into smaller sections based on the specified chunk size and overlap.
# The result is stored in `splits`, which will be a list of text chunks ready for further processing.
splits = text_splitter.split_documents(web_pages)

In [6]:
# Initialize the HuggingFaceEmbeddings class with a specific transformer model.
# The chosen model, 'sentence-transformers/paraphrase-MiniLM-L6-v2', is optimized for paraphrasing tasks,
# making it suitable for generating embeddings that capture the semantics of similar phrases effectively.

# `model_kwargs` is used to specify additional settings for the model loading process.
# Here, {'device': 'cpu'} forces the model to run on the CPU instead of a GPU.

# `encode_kwargs` contains parameters that affect the encoding process.
# Setting {'normalize_embeddings': True} ensures that the embeddings are normalized.
# Normalization, in this context, means scaling the embedding vectors to have a unit norm.
# This standardization often helps in improving the performance in similarity calculations by making
# the distance measures (like cosine similarity) focus strictly on the angle between vectors rather than their magnitude.

device = "cpu"
embeddings = HuggingFaceEmbeddings(
    model_name='sentence-transformers/paraphrase-MiniLM-L6-v2',
    model_kwargs={'device':device},
    encode_kwargs={'normalize_embeddings': True}
)

<ipython-input-6-f3ff04529717>:15: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(
/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/sett

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.73k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [7]:
# Initialize a Chroma vector store using the previously generated document splits and embeddings.
# `Chroma.from_documents` is a method used to create a vector store by processing a collection of documents,
# each represented by a segment of text from the `splits` list. The text segments are converted into
# vector embeddings using the `embeddings` object.

# `documents`: This parameter takes the list of document segments (`splits`) that need to be embedded.
# These segments were previously created by splitting larger documents into smaller, manageable chunks.

# `embedding`: This parameter receives the `embeddings` object, which specifies the model and settings
# used to generate embeddings. The embeddings are created using the 'sentence-transformers/paraphrase-MiniLM-L6-v2'
# model, which provides high-quality semantic embeddings for the text segments.

# The result, `vector_store`, is an object that holds a collection of vector embeddings corresponding to the
# input documents. This vector store can be used in various applications like semantic search, document similarity,
# and other NLP tasks that require comparing the content of different texts based on their vectorized forms.

# try:
#   if vector_store :
#     pprint("Cleaning the existing db")
#     vector_store.delete_collection()
# except NameError:
#     print("WARN: vector_store not defined")

vector_store = FAISS.from_documents(documents=splits, embedding=embeddings)
pprint(f"Vector store has {len(vector_store.index_to_docstore_id)} chunks, across {len(web_pages)} web pages.")

'Vector store has 107 chunks, across 2 web pages.'


In [8]:
# Define a list of questions related to the content of the documents loaded and split earlier.
# These questions are intended to be answered by referencing the textual content from those documents.
questions = [
    "Who is Kunal Sevak?",
    "What are his key strenghts and weakness?"
]

# Convert the vector store into a retriever capable of fetching relevant document segments based on a query.
# `search_kwargs={"k": 10}` configures the retriever to return the top 10 most relevant document segments for a given query.
retriever = vector_store.as_retriever(search_kwargs={"k": 10})

# Define a chat prompt template that formats the context and question for the language model.
# This template organizes the input so that the context is clearly separated from the question,
# ensuring that the model can distinguish between the provided background information and what it needs to answer.
prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:
[CONTEXT]:
{context}

[QUESTUIN]
{question}""")

# Function to assemble the context from retrieved document chunks.
# It takes retrieved chunks, extracts the page content from each, and joins them with double newlines.
# This formatting helps maintain a clear separation between different pieces of context.
def assemble_context(chunks):
    context = "\n\n".join([chunk.page_content for chunk in chunks])
    return context

# Define a processing chain that orchestrates the flow from context retrieval to response generation.
# The chain is composed of several steps:
# 1. `retriever | assemble_context`: Retrieves document chunks and assembles them into a single context string.
# 2. `RunnablePassthrough()`: Passes the question as is.
# 3. `prompt | llm | StrOutputParser()`: Formats the prompt, invokes the language model, and parses the string output.
chain = (
    {"context": retriever | assemble_context, "question": RunnablePassthrough()}
    | prompt | llm | StrOutputParser()
)

# Iterate over each question, invoke the chain with the question to get an answer, and print the results.
# The process retrieves context relevant to each question, formats it with the question, and generates an answer.
for question in questions:
    ans = chain.invoke(question)
    pprint({"Q)": question, "A)": ans}, sort_dicts=False)


{'Q)': 'Who is Kunal Sevak?',
 'A)': 'According to the provided context, Kunal Sevak is a data science '
       'professional with over 9 years of experience across various sectors. '
       'He holds a BTech-IT degree, PGDM-Predictive Analytics, and is a CSPOÂ® '
       '- Scispot (YC S21). He has worked as a Sr. Data Professional at Samos '
       'Insurance, Research Assistant at Smart Centre, Conestoga Applied '
       'Research, Senior Technology Specialist at Alltech Group, and Sr. '
       'Systems Engineer at Tata Consultancy Services.'}
{'Q)': 'What are his key strenghts and weakness?',
 'A)': "Based on the provided context, Kunal Sevak's key strengths appear to "
       'be:\n'
       '\n'
       '1. Data Analytics: As a BTech-IT and PGDM-Predictive Analytics '
       'graduate, he likely has strong skills in data analysis, visualization, '
       'and predictive modeling.\n'
       '2. Full Stack Development: He is proficient in building out visually '
       'appealing and 